**Notes before running**:
- Allow Internet access and set GPU runtime.
- If your repo is on GitHub, set `REPO_URL` below; otherwise upload the repo to Drive and set `DRIVE_REPO_PATH`.
- This notebook provides two inference options: `transformers+bitsandbytes` (recommended) and `vLLM` (optional, heavier).

In [ ]:
# 1) Install core dependencies (may take a few minutes)
# Adjust versions if your project requires specific ones.
import os
print('Installing packages...')
# Install main libs; vLLM is optional (comment/uncomment as needed)
!pip -q install --upgrade pip
!pip -q install transformers accelerate bitsandbytes safetensors sentencepiece huggingface_hub datasets wandb pandas scipy
# If you want vLLM on Colab (optional, can be heavy): uncomment next line
# !pip -q install vllm
print('Done')

In [ ]:
# 2) Mount Google Drive (optional but recommended to persist caches & results)
from google.colab import drive
drive.mount('/content/drive')
# Create directories for HF cache and model offload on Drive to avoid re-downloads
import os
HF_DIR = '/content/drive/MyDrive/hf_cache'
OFFLOAD_DIR = '/content/drive/MyDrive/hf_offload'
os.makedirs(HF_DIR, exist_ok=True)
os.makedirs(OFFLOAD_DIR, exist_ok=True)
os.environ['HF_HOME'] = HF_DIR
os.environ['TRANSFORMERS_CACHE'] = HF_DIR
os.environ['XDG_CACHE_HOME'] = HF_DIR
os.environ['HF_DATASETS_CACHE'] = HF_DIR
os.environ['ACCELERATE_STATE_DIR'] = OFFLOAD_DIR
print('HF cache ->', HF_DIR)

In [ ]:
# 2.5) Set a Drive output directory for results and checkpoints
import os
DRIVE_OUTPUT_DIR = '/content/drive/MyDrive/subliminal_results'
os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
os.environ['DRIVE_OUTPUT_DIR'] = DRIVE_OUTPUT_DIR
print('DRIVE_OUTPUT_DIR ->', DRIVE_OUTPUT_DIR)
# Optionally set HF cache/offload to Drive (already set above)

In [ ]:
# 3) Clone or copy your project into /content/SubliminalLearning
# Option A: clone from GitHub (set REPO_URL)
REPO_URL = 'https://github.com/GabraxVolz1/SubliminalLearning.git'  # e.g. 'https://github.com/yourname/SubliminalLearning.git'
DRIVE_REPO_PATH = '/content/drive/MyDrive/SubliminalLearning'  # if you uploaded repo to Drive
import os, shutil
TARGET = '/content/SubliminalLearning'
if os.path.exists(TARGET):
    print('Project already present at', TARGET)
elif REPO_URL:
    print('Cloning from', REPO_URL)
    !git clone --depth 1 {REPO_URL} {TARGET}
elif os.path.exists(DRIVE_REPO_PATH):
    print('Copying from Drive', DRIVE_REPO_PATH)
    shutil.copytree(DRIVE_REPO_PATH, TARGET)
else:
    print('No REPO_URL and DRIVE_REPO_PATH not found. Please set one and rerun this cell.')
print('Project directory:', os.path.exists(TARGET) and TARGET or 'NOT FOUND')

In [ ]:
# 4) Install project (editable) and any extra requirements if available
import os
PROJECT_DIR = '/content/SubliminalLearning'
if os.path.exists(PROJECT_DIR):
    %cd /content/SubliminalLearning
    # Prefer requirements.txt or pyproject-based install if present
    if os.path.exists('requirements.txt'):
        print('Installing requirements.txt')
        !pip -q install -r requirements.txt
    elif os.path.exists('pyproject.toml'):
        print('Installing project in editable mode (pyproject.toml found)')
        !pip -q install -e .
    else:
        print('No requirements.txt or pyproject.toml found — ensure dependencies are installed manually')
    %cd /content
else:
    print('Project dir not found; cannot install project')

In [ ]:
# 5) Set HF token (so private models can be downloaded)
from getpass import getpass
token = getpass('Enter your Hugging Face token (leave blank to skip): ')
if token:
    import os
    os.environ['HUGGINGFACE_HUB_TOKEN'] = token
    !huggingface-cli login --token {token}  # optional; helps with rate limits
else:
    print('Skipping token set; public models only.')

In [ ]:
# 6) Check GPU and memory (important before loading models)
import torch, subprocess
print('Torch version:', torch.__version__)
print('CUDA available:', torch.cuda.is_available())
if torch.cuda.is_available():
    try:
        print('Device:', torch.cuda.get_device_name(0))
    except Exception as e:
        print('Device name error:', e)
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv

## Inference options
Choose one of the cells below depending on model runtime preference and GPU capacity. If you have A100 (>=40GB) try `vLLM` or large models; otherwise use `transformers+bitsandbytes` 8-bit for 7B models.

In [ ]:
# 7A) Recommended: Load a 7B model with transformers + bitsandbytes (fast to try)
# This attempts an 8-bit load using device_map='auto' and requires bitsandbytes installed.
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
MODEL_NAME = 'mistralai/Mistral-7B-Instruct-v0.1'
try:
    print('Loading tokenizer...')
    tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)
    print('Loading model in 8-bit (this may take a minute)...')
    model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map='auto', load_in_8bit=True, trust_remote_code=True)
    gen = pipeline('text-generation', model=model, tokenizer=tokenizer, device_map='auto')
    print(gen('Write a concise greeting.', max_new_tokens=30)[0]['generated_text'])
except Exception as e:
    print('Error loading 8-bit model:')
    import traceback
    traceback.print_exc()

In [ ]:
# 7B) Optional: Start vLLM server (only if you installed vllm and have enough GPU memory)
# Uncomment and run if you want to try vLLM serving.
# Note: vLLM can require a lot of GPU RAM; prefer A100.
# !vllm serve mistralai/Mistral-7B-Instruct-v0.1 --tokenizer-mode mistral &
print('If you installed vLLM and have sufficient GPU memory, run the vLLM serve command in a terminal cell (commented).')

In [ ]:
# 8) Run a small end-to-end pilot of the project (adjust args for your experiment)
# This runs your repo's scripts with small sizes so you can verify the pipeline works on Colab.
import os, subprocess
PROJECT_DIR = '/content/SubliminalLearning'
if not os.path.exists(PROJECT_DIR):
    print('Project not available at', PROJECT_DIR)
else:
    %cd /content/SubliminalLearning
    # Example: generate a small teacher set (10) and checkpoint to Drive
    try:
        print('Generating teacher conversations (n=10)...')
        !python scripts/generate_teacher_conversations.py --count 10 --turns 1 --out {DRIVE_OUTPUT_DIR}/teachers_sample.jsonl
    except Exception as e:
        print('Teacher generation failed:', e)
    # Example: run a tiny student roleplay (limit 5) and save outputs to Drive
    try:
        print('Running student roleplay (pilot)...')
        !python scripts/run_student_roleplay.py --in {DRIVE_OUTPUT_DIR}/teachers_sample.jsonl --out {DRIVE_OUTPUT_DIR}/student_pilot.jsonl --limit 5 --batch-size 2
    except Exception as e:
        print('Student roleplay failed:', e)
    # Example: run ablation driver with tiny settings and save to Drive (uses DRIVE_OUTPUT_DIR by default)
    try:
        print('Running ablation driver (pilot)...')
        !python scripts/ablation_driver.py --teacher {DRIVE_OUTPUT_DIR}/teachers_sample.jsonl --limit 10 --out-dir {DRIVE_OUTPUT_DIR}/ablation_pilot
    except Exception as e:
        print('Ablation driver failed:', e)
    %cd /content

## Scaling to full experiment
- For large runs (e.g., 400 conversations) use Colab Pro+ A100 or a rented VM with a large GPU to avoid OOM and session time limits.
- Save intermediate outputs to Drive (`/content/drive/MyDrive/...`) to avoid losing progress on disconnects.
- Use `--num`/`--n_students`/`--n_repeats` parameters in your scripts to scale; monitor GPU memory with `!nvidia-smi`.